In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

# load the Tiny Shakespeare dataset
dataset, info = tfds.load('tiny_shakespeare', with_info=True, as_supervised=False)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incomplete9TW4X5/tiny_shakespeare-train.tfrecord*..…

Generating validation examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incomplete9TW4X5/tiny_shakespeare-validation.tfreco…

Generating test examples...:   0%|          | 0/1 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/tiny_shakespeare/1.0.0.incomplete9TW4X5/tiny_shakespeare-test.tfrecord*...…

Dataset tiny_shakespeare downloaded and prepared to /root/tensorflow_datasets/tiny_shakespeare/1.0.0. Subsequent calls will reuse this data.


In [2]:
# get the text from the dataset
text = next(iter(dataset['train']))['text'].numpy().decode('utf-8')

# create a mapping from unique characters to indices
vocab = sorted(set(text))
char2idx = {char: idx for idx, char in enumerate(vocab)}
idx2char = np.array(vocab)

# numerically represent the characters
text_as_int = np.array([char2idx[c] for c in text])

# create training examples and targets
seq_length = 100
examples_per_epoch = len(text) // (seq_length + 1)

# create training sequences
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length + 1, drop_remainder=True)

In [3]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [4]:
# batch size and buffer size
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

In [7]:
# length of the vocabulary
vocab_size = len(vocab)

# the embedding dimension
embedding_dim = 256

# number of RNN units
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True, stateful=True, recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model
model = build_model(vocab_size,embedding_dim,rnn_units, BATCH_SIZE)

In [8]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

In [9]:
import os

# directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

# train the model
EPOCHS = 10
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
155/155 [==============================] - 1225s 8s/step - loss: 2.6506
Epoch 2/10
155/155 [==============================] - 1184s 8s/step - loss: 1.9349
Epoch 3/10
155/155 [==============================] - 1181s 8s/step - loss: 1.6805
Epoch 4/10
155/155 [==============================] - 1169s 8s/step - loss: 1.5409
Epoch 5/10
155/155 [==============================] - 1175s 8s/step - loss: 1.4552
Epoch 6/10
155/155 [==============================] - 1179s 8s/step - loss: 1.3941
Epoch 7/10
155/155 [==============================] - 1176s 8s/step - loss: 1.3473
Epoch 8/10
155/155 [==============================] - 1172s 8s/step - loss: 1.3089
Epoch 9/10
155/155 [==============================] - 1182s 8s/step - loss: 1.2724
Epoch 10/10
155/155 [==============================] - 1194s 8s/step - loss: 1.2394


In [10]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [11]:
def generate_text(model, start_string):
    num_generate = 1000

    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))
print(generate_text(model, start_string = u"QUEEN: So, let's end this"))

QUEEN: So, let's end this Apter, plead.

POMPEY:
I best derived to him; I wepm itself as liss
Are an other the slight to tell it. Whot as the foils of it?

QUEEN:
Nimbut your violence hath much suit impatient.

BUCKINGHAM:
As if how I myself in heaven. Come, come on,
If thou on the haply died, that lives may do you toad
That born thus grows our willing: if you spoke
Proud Anfortathou, yield us not. verming to that slarped?

SAMPSON:
Away, britting here: though wild, he spokes
As crainSLOND:
The telder love thy daughter wood, you shall kisply to ride,
Let him have had, and stand belongh'd,
What wait you do defand thy husband:
I will despair. I; by shuffly, and lies the read?

JOHN OF GAUNT:
Most offence, but here it does n grand.
How in them grace to ourdee-sophereing out.
What mulder-have I do come to be his very gradl?

DUKE VINCENTIO:
First, sir; that most morn perssition, tribunes!
Do Polfus will make sake your honour.

ISABELLA:
Do leass his cords, Within my most hour chosens,
That